In [22]:
#Ｉmport necessary packages
import torch
import torch.nn as nn
import numpy as np
from matplotlib import pyplot as plt
import xitorch
from xitorch.optimize import rootfinder

In [2]:
# Define MLP for potentials
class PP(nn.Module):
    # Constructor
    def __init__(self, NNs, input_dim = 1, output_dim = 1):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, NNs[0]), 
            nn.ELU(),
        )
        
        for i in range(len(NNs) - 1):
            self.fc.append(nn.Linear(NNs[i], NNs[i + 1]))
            self.fc.append(nn.ELU())
        
        self.fc.append(nn.Linear(NNs[-1], output_dim))
    
    # Forward function
    def forward(self, x):
        return self.fc(x)

# Data generation

In [3]:
from DataGeneration import generateSamples, genVVtt
import os

generating_flag = False
kwgs = {
    "beta" : [0.011, 0.016, 1. / 1.e1, 0.58], 
    "totalNofSeqs" : 1024 * 16, 
    "NofIntervalsRange" : [5, 11], 
    "VVRange" : [-10, 3], 
    "VVLenRange" : [8, 9], 
    "theta0" : 1., 
    "prefix" : "Trial1003", 
    "NofVVSteps" : 400, 
}

# Generate / load data
dataFile = "./data/" + kwgs["prefix"] + ".pt"

if generating_flag or not(os.path.isfile(dataFile)):
    print("Generating data")
    generateSamples(kwgs)

shit = torch.load(dataFile)
Vs = shit["Vs"]
thetas = shit["thetas"]
fs = shit["fs"]

# Stack data as
Vs = torch.stack(Vs)[:1000, :500]
thetas = torch.stack(thetas)[:1000, :500]
fs = torch.stack(fs)[:1000, :500]
ts = shit["ts"][:1000, :500]

/home/shengduo/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [192]:
# Now Vs and ts have fixed length
print("Vs.shape: ", Vs.shape)
print("thetas.shape: ", thetas.shape)
print("fs.shape: ", fs.shape)
print("ts.shape: ", ts.shape)

Vs.shape:  torch.Size([1000, 500])
thetas.shape:  torch.Size([1000, 500])
fs.shape:  torch.Size([1000, 500])
ts.shape:  torch.Size([1000, 500])


In [195]:
# Calculate Xs
Xs = torch.zeros(Vs.shape)
Xs[:, 1:] = torch.cumulative_trapezoid(Xs, ts)
print("Xs.shape: ", Xs.shape)

Xs.shape:  torch.Size([1000, 500])


# Defining NNs, for $W (V, \xi)$ and $D (V, \xi, \dot{\xi})$

In [5]:
# Specify dimension of xi
dim_xi = 4

# Specify NNs for W and D
NNs_W = [128, 128]
NNs_D = [256, 256]

kwgsPot = {
    "dim_xi" : dim_xi, 
    "NNs_W" : NNs_W, 
    "NNs_D" : NNs_D, 
}

# Calculate $f = \partial W / \partial V$, $\xi_{n+1}$ such that $\partial D / \partial \dot{\xi} + \partial W / \partial \xi = 0$

In [196]:
# Define class for training and calculating f
# Optimizer Adams
import torch.optim as optim

class PotentialsFric:
    # Initialization of W and D
    def __init__(self, kwgsPot):
        self.dim_xi = kwgsPot["dim_xi"]
        self.NNs_W = kwgsPot["NNs_W"]
        self.NNs_D = kwgsPot["NNs_D"]
        self.W = PP(NNs_W, input_dim = 1 + dim_xi, output_dim = 1)
        # self.D = PP(NNs_D, input_dim = 1 + 2 * dim_xi, output_dim = 1)
        self.optim_W = optim.Adam(self.W.parameters(), lr=0.001)
        # self.optim_D = optim.Adam(self.D.parameters(), lr=0.001)
    
    # Calculate f 
    def calf(self, x, t):
        # Initialize Vs
        batch_size = x.shape[0]
        time_steps = x.shape[1]
        xi0 = torch.zeros([batch_size, self.dim_xi], requires_grad=True)
        # xis[:, :, :] = 1. 
        
        # List of fs
        list_fs = []
        list_xis = [xi0]
        
        # Loop through time steps
        for idx in range(x.shape[1]):
            # f = \partial W / \partial V
            X_W = torch.concat([x[:, idx].reshape([-1, 1]), list_xis[-1]], dim = 1).requires_grad_()
            W = torch.sum(self.W(X_W))
            
            this_piece = torch.autograd.grad(outputs=W, inputs=X_W, create_graph=True)[0]
            list_fs.append(this_piece[:, 0:1])
            
            # Solve for \dot{\xi} + \partial W / \partial \xi = 0
            dWdXi = this_piece[:, 1:]
            
            # XiDot = -dWdXi
            if idx < x.shape[1] - 1:
                xiNext = list_xis[-1] - dWdXi * (t[:, idx + 1:idx + 2] - t[:, idx:idx + 1]) 
                list_xis.append(xiNext)
        self.fs = torch.concat(list_fs, dim=1)
            

# Define Loss function, training function, dataloaders

In [198]:
# Define loss functions given fs_targ, fs. 
def Loss(fs_targ, fs, ts, p = 2):
    return torch.sum(torch.pow(torch.trapz((fs_targ - fs) ** p, ts, dim = 1), 1. / p))

# Training for one epoch
def train1Epoch(data_loader, loss_fn, myPot):
    # Record of losses for each batch
    Losses = []
    
    # Enumerate over data_loader
    for idx, (Vs, ts, fs_targ) in enumerate(data_loader):
        # Refresh the optimizers
        myPot.optim_W.zero_grad()
        myPot.optim_D.zero_grad()
        
        # Compute loss
        myPot.calf(Vs, ts)
        loss = loss_fn(fs_targ, myPot.fs, ts)
        Losses.append(loss)
        
        # Update the model parameters
        loss.backward()
        myPot.optim_W.step()
        
        if hasattr(myPot, 'optim_D'):
            myPot.optim_D.step()
        
    return sum(Losses) / len(data_loader.dataset)


In [199]:
# Initialize dataloaders
from torch.utils.data import TensorDataset, DataLoader
AllData = TensorDataset(
    Xs, 
    ts, 
    fs
)

train_len = int(len(Vs) * 0.8)
test_len = len(Vs) - train_len
trainDataset, testDataset = torch.utils.data.random_split(AllData, [train_len, test_len])

# Training data loader
training_batch_size = 64 #1024
trainDataLoader = DataLoader(
    trainDataset,
    batch_size = training_batch_size,
    shuffle = True,
#    num_workers = 16,
    collate_fn = None,
    pin_memory = False,
)

# Testing data loader
testing_batch_size = 16 # 256
testDataLoader = DataLoader(
    testDataset,
    batch_size = testing_batch_size,
    shuffle = True,
#    num_workers = 16,
    collate_fn = None,
    pin_memory = False,
)

In [187]:
# Get a test case for potentials
myWD = PotentialsFric(kwgsPot)

# Train for one epoch
train1Epoch(trainDataLoader, Loss, myWD)

RuntimeError: One of the differentiated Tensors appears to not have been used in the graph. Set allow_unused=True if this is the desired behavior.

In [191]:
myWD.shitAssXiDot.grad_fn

In [ ]:
a = torch.zeros([5,5])
a.requires_grad

In [86]:
myWD.fs

tensor([[0.0361, 0.0361, 0.0361,  ..., 0.0359, 0.0359, 0.0359],
        [0.0338, 0.0338, 0.0338,  ..., 0.0362, 0.0362, 0.0362],
        [0.0362, 0.0362, 0.0362,  ..., 0.0362, 0.0362, 0.0362],
        ...,
        [0.0362, 0.0362, 0.0362,  ..., 0.0362, 0.0362, 0.0362],
        [0.0362, 0.0362, 0.0362,  ..., 0.0362, 0.0362, 0.0362],
        [0.0362, 0.0362, 0.0362,  ..., 0.0362, 0.0362, 0.0362]])

In [30]:
myWD.fs.shape

torch.Size([16384, 4000])

# Demo: Try taking gradients w.r.t. inputs, this works

In [3]:
# Construct my NN with initialized default parameters 
NNs = [16, 16]
input_dim = 4
output_dim = 1

myPP = PP(NNs, input_dim, output_dim)

In [23]:
# Try taking gradients w.r.t. the inputs
x = torch.tensor([[1., 2., -1., 3.]], requires_grad=True)
y = torch.sum(myPP(x))
dydx = torch.autograd.grad(outputs=y, inputs=x, retain_graph=True)[0]

# Show values and gradients
print("y: ", y)
print("dydx: ", dydx)

y:  tensor(-0.1882, grad_fn=<SumBackward0>)
dydx:  tensor([[ 0.0650, -0.0497, -0.0758, -0.0635]])


In [22]:
# Try taking gradients w.r.t. the inputs
x = torch.tensor([[1., 2., -1., 3.], [-1., -3., 2., 5.]], requires_grad=True)
y = torch.sum(myPP(x))
dydx = torch.autograd.grad(outputs=y, inputs=x, retain_graph=True)[0]

# Show values and gradients
print("y: ", y)
print("dydx: ", dydx)

y:  tensor(-0.3059, grad_fn=<SumBackward0>)
dydx:  tensor([[ 0.0650, -0.0497, -0.0758, -0.0635],
        [ 0.0623, -0.0945, -0.0193, -0.0369]])


In [20]:
dydx[0]

tensor([[ 0.0650, -0.0497, -0.0758, -0.0635],
        [ 0.0623, -0.0945, -0.0193, -0.0369]])

In [21]:
y

tensor(-0.3059, grad_fn=<SumBackward0>)